In [314]:
# The code was removed by Watson Studio for sharing.

# Relevant Places for Cinnabon Bakehouse

Analysis of buildings within Vladivostok city to see if they are suitable for opening __Cinnabon__ bakehouses there.

Above all, load necessary libraries, set the <code>CLIENT_ID</code> and the <code>CLIENT_SECRET</code> and ect.

In [315]:
# The code was removed by Watson Studio for sharing.

## Place evaluation criteria

I will drop some criteria because of their estimation difficulty, writing code time-intensivity and the lack of data.

+ __Bakehouses should be placed in the places with high capacitintense pedestrian flows:__
    - major malls
    - cinema

+ __Location of the best place:__
    - the first floor of malls, next to the entrance
    - having a close parking place, not far than 20 meters.
    - the first line of houses

## Get a list of the major malls

In [316]:
mall_query = 'Торговый центр'

Define the URL for find the malls.

In [317]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, near, VERSION, mall_query)
url

'https://api.foursquare.com/v2/venues/search?client_id=5KPM10HKO1V1ZA3NJI5Y5YQERX2CWFO13LD05YAQ1JPLONS5&client_secret=1VMF22DJYOZC2AZ3N200OTPF1FVHZNBUYAOSZI4H01CVBNNT&near=Владивосток&v=20180604&query=Торговый центр'

In [318]:
results = requests.get(url).json()
#results

'Nursery School'? Hmm, it is not exactly that I am, looking for. As We can see, Foursquare search engine works not enough accuracy.  

### Data cleaning

In [319]:
#Assign relevant part of JSON to venues.
venues = results['response']['venues']

ind_mall = set()
for ind, row in enumerate(venues):
    if row['categories'][0]['shortName'] == 'Mall':
        ind_mall.add(ind)
        
ind_city = set()
for ind, row in enumerate(venues):
    if 'city' in row['location'].keys():
        if 'Владивосток' in row['location']['city']:
            ind_city.add(ind)
            
valid_malls = ind_mall.intersection(ind_city)
valid_malls = list(valid_malls)
valid_malls

[1, 2, 3, 6, 9, 10, 11, 13, 22]

Get relevant part of <code>.json</code> and transform it into a <code>pandas</code> DataFrame.

In [320]:
#Tranform venues into a dataframe.
df_malls = json_normalize(venues)

#Apply a valid_malls mask to df_malls DataFrame.
df_malls = df_malls.iloc[valid_malls, :]
df_malls.reset_index(inplace=True, drop=True)
df_malls

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4e437b54d4c02cf15f569f74,NaN,RU,Владивосток,Россия,NaN,"[Владивосток, Россия]","[{'label': 'display', 'lat': 43.11667531484311...",43.116675,131.882463,NaN,Приморский край,"Торговый центр ""Золотой Рог""",v-1580549481
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4d92b3aef10ea093077f191e,NaN,RU,Владивосток,Россия,NaN,"[Владивосток, Россия]","[{'label': 'display', 'lat': 43.13333257579817...",43.133333,131.899469,NaN,Приморский край,Первореченский Торговый Центр,v-1580549481
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,584b433391936a1570de70b3,Шилкинская д.42,RU,Владивосток,Россия,NaN,"[Шилкинская д.42, 690066, Владивосток, Россия]","[{'label': 'display', 'lat': 43.119311, 'lng':...",43.119311,131.932502,690066,Приморский край,"Торговый центр ""Шилка""",v-1580549481
3,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4f961eebe4b0e8af220bce50,"Русская ул., 59/2",RU,Владивосток,Россия,NaN,"[Русская ул., 59/2, 690105, Владивосток, Россия]","[{'label': 'display', 'lat': 43.16926377142994...",43.169264,131.924898,690105,Приморский край,Торговый Центр «Заря»,v-1580549481
4,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4e2b8ab1483bb05f05a05d75,"Светланская ул., 29",RU,Владивосток,Россия,Океанский просп.,"[Светланская ул., 29 (Океанский просп.), 69009...","[{'label': 'display', 'lat': 43.11616328803306...",43.116163,131.885856,690091,Приморский край,ТК «Центральный»,v-1580549481
5,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4e27cf8b483baa86514f83ef,"ул.Луговая, 21а",RU,Владивосток,Россия,NaN,"[ул.Луговая, 21а, 690005, Владивосток, Россия]","[{'label': 'display', 'lat': 43.11140805720977...",43.111408,131.936717,690005,Приморский край,Пешеходно-Торговый комплекс на Луговой,v-1580549481
6,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4e4b8d23aeb72792c33ac8ef,"Семёновская ул., 12",RU,Владивосток,Россия,NaN,"[Семёновская ул., 12, Владивосток, Россия]","[{'label': 'display', 'lat': 43.1186449390325,...",43.118645,131.883044,NaN,Приморский край,ТЦ «Родина»,v-1580549481
7,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4c6ce4e61fb2a14369bdfce6,"Семёновская ул., 15",RU,Владивосток,Россия,NaN,"[Семёновская ул., 15, 690091, Владивосток, Рос...","[{'label': 'display', 'lat': 43.11883930771354...",43.118839,131.884034,690091,Приморский край,Clover House,v-1580549481
8,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4d88610b4757721e7d468c69,"Светланская ул., 106",RU,Владивосток,Россия,ул. Капитана Шефнера,"[Светланская ул., 106 (ул. Капитана Шефнера), ...","[{'label': 'display', 'lat': 43.11207926680047...",43.112079,131.917394,690001,Приморский край,ТЦ Авангард,v-1580549481


In [321]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_malls.columns if col.startswith('location.')] + ['id']
df_malls = df_malls.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_malls['categories'] = df_malls.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_malls.columns = [column.split('.')[-1] for column in df_malls.columns]
df_malls = df_malls[['name', 'categories', 'lat', 'lng', 'id']]

df_malls

,name,categories,lat,lng,id
0,"Торговый центр ""Золотой Рог""",Shopping Mall,43.116675,131.882463,4e437b54d4c02cf15f569f74
1,Первореченский Торговый Центр,Shopping Mall,43.133333,131.899469,4d92b3aef10ea093077f191e
2,"Торговый центр ""Шилка""",Shopping Mall,43.119311,131.932502,584b433391936a1570de70b3
3,Торговый Центр «Заря»,Shopping Mall,43.169264,131.924898,4f961eebe4b0e8af220bce50
4,ТК «Центральный»,Shopping Mall,43.116163,131.885856,4e2b8ab1483bb05f05a05d75
5,Пешеходно-Торговый комплекс на Луговой,Shopping Mall,43.111408,131.936717,4e27cf8b483baa86514f83ef
6,ТЦ «Родина»,Shopping Mall,43.118645,131.883044,4e4b8d23aeb72792c33ac8ef
7,Clover House,Shopping Mall,43.118839,131.884034,4c6ce4e61fb2a14369bdfce6
8,ТЦ Авангард,Shopping Mall,43.112079,131.917394,4d88610b4757721e7d468c69


## Get a list of the cinema

In [322]:
cinema_query = 'Кинотеатр'

Define the URL for find the cinema.

In [323]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, near, VERSION, cinema_query)
url

'https://api.foursquare.com/v2/venues/search?client_id=5KPM10HKO1V1ZA3NJI5Y5YQERX2CWFO13LD05YAQ1JPLONS5&client_secret=1VMF22DJYOZC2AZ3N200OTPF1FVHZNBUYAOSZI4H01CVBNNT&near=Владивосток&v=20180604&query=Кинотеатр'

In [324]:
results = requests.get(url).json()
#results

### Data cleaning

In [325]:
#Assign relevant part of JSON to venues.
venues = results['response']['venues']

ind_cinema = set()
for ind, row in enumerate(venues):
    if row['categories'][0]['shortName'] == 'Movie Theater':
        ind_mall.add(ind)
        
ind_city = set()
for ind, row in enumerate(venues):
    if 'city' in row['location'].keys():
        if 'Владивосток' in row['location']['city']:
            ind_city.add(ind)
            
valid_cinema = ind_mall.intersection(ind_city)
valid_cinema = list(valid_cinema)
valid_cinema

[0, 1, 3, 4, 5, 6, 7, 8]

In [326]:
#Tranform venues into a dataframe.
df_cinema = json_normalize(venues)

#Apply a valid_malls mask to df_malls DataFrame.
df_cinema = df_cinema.iloc[valid_cinema, :]
df_cinema.reset_index(inplace=True, drop=True)
df_cinema

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",False,51b32091498e954e6c465027,Набережная ул.,RU,Владивосток,Россия,NaN,"[Набережная ул., Владивосток, Россия]","[{'label': 'display', 'lat': 43.11759998870515...",43.117600,131.878222,NaN,Приморский край,Кинотеатр под открытым небом,v-1580549481
1,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",False,4f6bf0dce4b068929e154d5d,"Трамвайная ул., 16",RU,Владивосток,Россия,пл. Луговая,"[Трамвайная ул., 16 (пл. Луговая), 690034, Вла...","[{'label': 'display', 'lat': 43.11197102091505...",43.111971,131.937990,690034,Приморский край,Кинотеатр «Владивосток»,v-1580549481
2,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",False,53255a8f498ed5af59489dd0,Батарейная,RU,Владивосток,Россия,NaN,"[Батарейная, Владивосток, Россия]","[{'label': 'display', 'lat': 43.120622, 'lng':...",43.120622,131.876021,NaN,Приморский край,7Д Кинотеатр,v-1580549481
3,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",False,4c70d4db9375a0932f5b0837,"ул. Набережная, 3",RU,Владивосток,Россия,ул. Тигровая,"[ул. Набережная, 3 (ул. Тигровая), 690091, Вла...","[{'label': 'display', 'lat': 43.11662153797272...",43.116622,131.877880,690091,Приморский край,Океан IMAX,v-1580549481
4,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",False,4c68c79735d3be9ab1d31d06,"Светланская ул., 31",RU,Владивосток,Россия,NaN,"[Светланская ул., 31, 690091, Владивосток, Рос...","[{'label': 'display', 'lat': 43.11575221665022...",43.115752,131.886885,690091,Приморский край,Уссури,v-1580549481
5,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",False,4de0303bfa76425c5412fd69,"просп. 100-летия Владивостока, 103",RU,Владивосток,Россия,NaN,"[просп. 100-летия Владивостока, 103, 690039, В...","[{'label': 'display', 'lat': 43.16248281279498...",43.162483,131.911358,690039,Приморский край,Иллюзион,v-1580549481
6,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",False,4c4a55c69c8d2d7f7dc75d69,"ул. Добровольского, 5",RU,Владивосток,Россия,NaN,"[ул. Добровольского, 5, 690092, Владивосток, Р...","[{'label': 'display', 'lat': 43.09219390373713...",43.092194,131.973069,690092,Приморский край,Галактика,v-1580549481
7,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",False,4ecb62c093adb4bd1ad84427,"Русская ул., 41",RU,Владивосток,Россия,NaN,"[Русская ул., 41, 690105, Владивосток, Россия]","[{'label': 'display', 'lat': 43.16949724662987...",43.169497,131.922403,690105,Приморский край,Нептун,v-1580549481


In [327]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_cinema.columns if col.startswith('location.')] + ['id']
df_cinema = df_cinema.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_cinema['categories'] = df_cinema.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_cinema.columns = [column.split('.')[-1] for column in df_cinema.columns]
df_cinema = df_cinema[['name', 'categories', 'lat', 'lng', 'id']]

df_cinema

,name,categories,lat,lng,id
0,Кинотеатр под открытым небом,Movie Theater,43.117600,131.878222,51b32091498e954e6c465027
1,Кинотеатр «Владивосток»,Movie Theater,43.111971,131.937990,4f6bf0dce4b068929e154d5d
2,7Д Кинотеатр,Movie Theater,43.120622,131.876021,53255a8f498ed5af59489dd0
3,Океан IMAX,Multiplex,43.116622,131.877880,4c70d4db9375a0932f5b0837
4,Уссури,Multiplex,43.115752,131.886885,4c68c79735d3be9ab1d31d06
5,Иллюзион,Multiplex,43.162483,131.911358,4de0303bfa76425c5412fd69
6,Галактика,Movie Theater,43.092194,131.973069,4c4a55c69c8d2d7f7dc75d69
7,Нептун,Movie Theater,43.169497,131.922403,4ecb62c093adb4bd1ad84427


Delete the firs row of <code>df_cinema</code>, because this cinema has no its own building according to its name "Кинотеатр под открытым небом" - "Open Air Cinema".

In [328]:
df_cinema.drop([0])

,name,categories,lat,lng,id
1,Кинотеатр «Владивосток»,Movie Theater,43.111971,131.937990,4f6bf0dce4b068929e154d5d
2,7Д Кинотеатр,Movie Theater,43.120622,131.876021,53255a8f498ed5af59489dd0
3,Океан IMAX,Multiplex,43.116622,131.877880,4c70d4db9375a0932f5b0837
4,Уссури,Multiplex,43.115752,131.886885,4c68c79735d3be9ab1d31d06
5,Иллюзион,Multiplex,43.162483,131.911358,4de0303bfa76425c5412fd69
6,Галактика,Movie Theater,43.092194,131.973069,4c4a55c69c8d2d7f7dc75d69
7,Нептун,Movie Theater,43.169497,131.922403,4ecb62c093adb4bd1ad84427


### Summary

Concatenating dataframes together.

In [329]:
df_places = pd.concat([df_malls, df_cinema])
df_places.reset_index(inplace=True, drop=True)
df_places.columns = ['Name', 'Categories', 'latitude', 'longitude', 'ID']
df_places

,Name,Categories,latitude,longitude,ID
0,"Торговый центр ""Золотой Рог""",Shopping Mall,43.116675,131.882463,4e437b54d4c02cf15f569f74
1,Первореченский Торговый Центр,Shopping Mall,43.133333,131.899469,4d92b3aef10ea093077f191e
2,"Торговый центр ""Шилка""",Shopping Mall,43.119311,131.932502,584b433391936a1570de70b3
3,Торговый Центр «Заря»,Shopping Mall,43.169264,131.924898,4f961eebe4b0e8af220bce50
4,ТК «Центральный»,Shopping Mall,43.116163,131.885856,4e2b8ab1483bb05f05a05d75
5,Пешеходно-Торговый комплекс на Луговой,Shopping Mall,43.111408,131.936717,4e27cf8b483baa86514f83ef
6,ТЦ «Родина»,Shopping Mall,43.118645,131.883044,4e4b8d23aeb72792c33ac8ef
7,Clover House,Shopping Mall,43.118839,131.884034,4c6ce4e61fb2a14369bdfce6
8,ТЦ Авангард,Shopping Mall,43.112079,131.917394,4d88610b4757721e7d468c69
9,Кинотеатр под открытым небом,Movie Theater,43.117600,131.878222,51b32091498e954e6c465027


In [330]:
address = 'Vladivostok'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude, longitude)

43.1150678 131.8855768


In [331]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) #Generate map centred around the 
                                                                       #Vladivostok city.

#Add the malls as blue circle markers and the cinema as red cirle markers.
for lat, lng, label, ctg in zip(df_places.latitude, df_places.longitude, df_places.Name, df_places.Categories):
    if 'Mall' in ctg:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            popup=label,
            fill = True,
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
    else:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='red',
            popup=label,
            fill = True,
            fill_color='red',
            fill_opacity=0.6
        ).add_to(venues_map)
    
#Display map.
venues_map

It was be trivial. Much harder will show relevant places in the first line of the houses. And it will be a real useful information.

## Die Hard

### Data Mining

In [332]:
try:
    import geopandas as gpd
except ModuleNotFoundError:
        !conda install geopandas
finally:
    try:
        import geopandas as gpd
    except ModuleNotFoundError:
        print('geopandas isn\'t installed.')

I was downloaded the <code>.shp.zip</code> file from [Geofabrik](https://download.geofabrik.de/russia/far-eastern-fed-district.html, "OpenStreetMap Data Extracts"). Then I did some magic with [Mapshaper](https://mapshaper.org/.In) with use <code>-filter 'name="Владивосток"'</code> on the main map, what cutted off another places, then I called <code>-clip my_part.shp</code> when I was in the another map, which I would like to cut off by using lately a recieved template. In contrast to 85 Gb of [the whole Earth](https://planet.openstreetmap.org/, "Planet OSM")  this 419 Mb of the Far Easter Federal District <code>.shp</code> file I downloaded is not so terribly big. Now, let load up the data and check it.

In [333]:
# The code was removed by Watson Studio for sharing.

,osm_id,code,fclass,name,geometry
0,25617751,7202,park,Сквер им. Суханова,"POLYGON ((131.89212 43.11855, 131.89232 43.118..."
1,26687466,7202,park,Адмиральский сквер,"POLYGON ((131.89044 43.11387, 131.89053 43.114..."
2,27152608,7202,park,,"POLYGON ((131.91059 43.10179, 131.91060 43.101..."
3,27420195,7202,park,Театральный сквер,"POLYGON ((131.89258 43.11454, 131.89272 43.114..."
4,27534478,7202,park,Сквер им. Невельского,"POLYGON ((131.89669 43.11303, 131.89671 43.113..."


There is not everyone class of land is suitable for bakehouse. We can't to locate bakery onto the administrative areas, onto military unit, cemetery and so on. Let show the set of different classes of the land.

In [334]:
print('Length of DataFrame is: ', len(df_landuse))
pd.unique(df_landuse['fclass'])

Length of DataFrame is:  8959


array(['park', 'forest', 'military', 'industrial', 'residential',
       'cemetery', 'scrub', 'grass', 'quarry', 'commercial', 'retail',
       'nature_reserve', 'allotments', 'heath', 'orchard', 'farm'],
      dtype=object)

In [335]:
usable_land = ['residential', 'commercial', 'retail'] #Yes, 'residential' too. I check it, in Mapshaper, it can
                                                      #be. 

print('Length of the DataFrame contains any class of land is: ', len(df_landuse))
df_usable = df_landuse[df_landuse.fclass.apply(lambda land: land in usable_land)]
df_usable.reset_index(inplace=True, drop=True)
df_usable.head()

Length of the DataFrame contains any class of land is:  8959


,osm_id,code,fclass,name,geometry
0,33389088,7203,residential,,"POLYGON ((131.91132 43.15505, 131.91133 43.155..."
1,44888532,7203,residential,,"POLYGON ((131.90832 43.15458, 131.90829 43.154..."
2,45061124,7203,residential,,"POLYGON ((131.88793 43.13638, 131.88798 43.136..."
3,45376149,7203,residential,,"POLYGON ((131.91396 43.10271, 131.91407 43.102..."
4,45784015,7209,commercial,,"POLYGON ((131.88375 43.11889, 131.88376 43.119..."


In [336]:
print('Length of the DataFrame contains usable land is: ', len(df_usable))

Length of the DataFrame contains usable land is:  603


Loading parking places data and buildings data. These data contains polygones each of them are represent parking areas and buildings perymetrs.

In [337]:
body = client_4b48aeaada904ad29e3d3d03072d749e.get_object(Bucket='courserads-donotdelete-pr-7suxmds10oqqm9',Key='Vladivostok_buildings.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )



df_buildings = gpd.read_file(body)
df_buildings.head()

,osm_id,code,fclass,name,type,geometry
0,23946365,1500,building,Океан,,"POLYGON ((131.87754 43.11645, 131.87755 43.116..."
1,24164985,1500,building,Морской Вокзал,,"POLYGON ((131.88222 43.11082, 131.88268 43.111..."
2,26537156,1500,building,Приморский драматический театр имени Максима Г...,,"POLYGON ((131.89305 43.11570, 131.89329 43.116..."
3,26686828,1500,building,Цирк,,"POLYGON ((131.90547 43.11662, 131.90552 43.116..."
4,26826883,1500,building,FESCO Hall,,"POLYGON ((131.87112 43.10551, 131.87117 43.105..."


In [338]:
print('Length of DataFrame is: ', df_buildings.shape[0])

Length of DataFrame is:  57604


Check unique types of buildings.

In [339]:
types = pd.unique(df_buildings['type'])
types

array(['', 'apartments', 'university', 'public', 'school', 'construction',
       'cathedral', 'warehouse', 'hospital', 'commercial', 'ruins',
       'industrial', 'service', 'kindergarten', 'detached', 'garage',
       'roof', 'garages', 'house', 'college', 'church', 'residential',
       'retail', 'dormitory', 'train_station', 'office', 'civic',
       'hangar', 'greenhouse', 'shed', 'hotel', 'bungalow', 'cс', 'kiosk',
       'chapel', 'toilets', 'non-residential', 'parking', 'barn', 'н',
       'dв', 'stadium', 'farm'], dtype=object)

Show names corresponding to each type of building.

In [340]:
for t in types[1:]:
    summary = df_buildings['name'][(df_buildings['type'] == t) & 
                                (df_buildings['name'] != '')]
    print(t + ':\n', pd.unique(summary), '\n')

apartments:
 ['Серая Лошадь' 'ТВ-Центр' 'Офис компании "Подряд"'
 'Мегабайт. Продажа и ремонт компьютеров и ноутбуков' 'Фрегат'
 'OOO "Belsu co LTD"' 'Алые паруса' 'Жилой комплекс «Театральный»'
 'OOO "VT Forwarding"' 'Жилой дом «Премьера»'
 'Жилой комплекс «Бригантина»'] 

university:
 ['ДВФУ. Школа естественных наук' 'Восточный Институт ДВФУ' 'ДВФУ'
 'Дальневосточный геологический институт ДВО РАН'
 'Тихоокеанский институт географии ДВО РАН'] 

public:
 ['Институт биологии моря ДВО РАН' 'Владивостокское музыкальное училище'
 'Дальневосточный энергетический техникум'
 'Краевой психоневрологический дом ребёнка'
 'Управление Россельхознадзора по Приморскому краю'
 'Главный корпус. Городская больница №4.Терапевтический'
 'Дальзаводская поликлинника (#4)' 'Отделение сестринского ухода'
 'Родильный Дом №1' 'Краевая стоматологическая поликлиника'
 'Мировые судьи Ленинского района'
 'Православный храм Святого апостола Андрея Первозванн'
 'Краевая детская психиатрическая больница' 'Детская шк

In [341]:
usable_buildings = ['', 'apartments', 'retail', 'hotel'] 

print('Length of the DataFrame contains any type of buildings is:', df_buildings.shape[0])
df_bakeries = df_buildings[df_buildings['type'].apply(lambda bld: bld in usable_buildings)]
df_bakeries.reset_index(inplace=True, drop=True)
df_bakeries.head()

Length of the DataFrame contains any type of buildings is: 57604


,osm_id,code,fclass,name,type,geometry
0,23946365,1500,building,Океан,,"POLYGON ((131.87754 43.11645, 131.87755 43.116..."
1,24164985,1500,building,Морской Вокзал,,"POLYGON ((131.88222 43.11082, 131.88268 43.111..."
2,26537156,1500,building,Приморский драматический театр имени Максима Г...,,"POLYGON ((131.89305 43.11570, 131.89329 43.116..."
3,26686828,1500,building,Цирк,,"POLYGON ((131.90547 43.11662, 131.90552 43.116..."
4,26826883,1500,building,FESCO Hall,,"POLYGON ((131.87112 43.10551, 131.87117 43.105..."


In [342]:
print(df_bakeries.shape[0])
pd.unique(df_bakeries['type'])

29101


array(['', 'apartments', 'retail', 'hotel'], dtype=object)

In [343]:
df_bakeries = df_bakeries.drop(['osm_id', 'code', 'fclass', 'name', 'type'], axis=1)
df_bakeries['centroid'] = df_bakeries.geometry.apply(lambda polygon: polygon.centroid)
df_bakeries.head()

,geometry,centroid
0,"POLYGON ((131.87754 43.11645, 131.87755 43.116...",POINT (131.87803 43.11650)
1,"POLYGON ((131.88222 43.11082, 131.88268 43.111...",POINT (131.88291 43.11138)
2,"POLYGON ((131.89305 43.11570, 131.89329 43.116...",POINT (131.89343 43.11599)
3,"POLYGON ((131.90547 43.11662, 131.90552 43.116...",POINT (131.90620 43.11612)
4,"POLYGON ((131.87112 43.10551, 131.87117 43.105...",POINT (131.87133 43.10507)


Bakeries must be placed only onto the suitable land. Filter bakeries by this condition with using <code>geopandas</code> method <code>.within(other)</code> for checking contenting one in other and atribute
<code>.centrod</code> which returns the centroid of polygon.

In [344]:
putin = lambda building: bool(df_usable.geometry.contains(building).sum())
df_bakeries = df_bakeries[df_bakeries.centroid.apply(putin)]

In [345]:
print('Length of the DataFrame contains useful class of buildings is:', df_bakeries.shape[0])

Length of the DataFrame contains useful class of buildings is: 13655


Check how many parking places contains this DataFrame.

In [346]:
len(df_buildings[df_buildings['type'] == 'parking'])

5

Actually, as a native citizen I know Vladivostok city has more parking. To find them is necessary load another DataFrame which holds only parking. Next to a bakehouse must has a parking not far than 20 meters.

In [347]:
body = client_4b48aeaada904ad29e3d3d03072d749e.get_object(Bucket='courserads-donotdelete-pr-7suxmds10oqqm9',Key='Vladivostok_traffic.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_parking = gpd.read_file(body)
df_parking.head()

,osm_id,code,fclass,name,geometry
0,47027380,5270,parking_bicycle,,"POLYGON ((131.89171 43.13051, 131.89171 43.130..."
1,47286823,5260,parking,,"POLYGON ((131.93530 43.11601, 131.93544 43.116..."
2,47288293,5260,parking,,"POLYGON ((131.93640 43.11494, 131.93641 43.114..."
3,47419448,5260,parking,,"POLYGON ((131.88875 43.11797, 131.88888 43.118..."
4,47425341,5260,parking,,"POLYGON ((131.88418 43.11831, 131.88420 43.118..."


In [348]:
pd.unique(df_parking.fclass)

array(['parking_bicycle', 'parking', 'marina', 'parking_multistorey',
       'fuel', 'dam', 'parking_underground'], dtype=object)

In [349]:
usable_parking = ['parking_bicycle', 'parking', 'parking_multistorey', 'parking_underground']

print('Length of the DataFrame contains any class of parking is: ', df_parking.shape[0])
df_parking = df_parking[df_parking['fclass'].apply(lambda fclass: fclass in usable_parking)]
df_parking.reset_index(inplace=True, drop=True)
df_parking.head()

Length of the DataFrame contains any class of parking is:  9036


,osm_id,code,fclass,name,geometry
0,47027380,5270,parking_bicycle,,"POLYGON ((131.89171 43.13051, 131.89171 43.130..."
1,47286823,5260,parking,,"POLYGON ((131.93530 43.11601, 131.93544 43.116..."
2,47288293,5260,parking,,"POLYGON ((131.93640 43.11494, 131.93641 43.114..."
3,47419448,5260,parking,,"POLYGON ((131.88875 43.11797, 131.88888 43.118..."
4,47425341,5260,parking,,"POLYGON ((131.88418 43.11831, 131.88420 43.118..."


In [350]:
pd.unique(df_parking.fclass)

array(['parking_bicycle', 'parking', 'parking_multistorey',
       'parking_underground'], dtype=object)

In [351]:
print('Length of the DataFrame contains useful class of parking is:', df_parking.shape[0])

Length of the DataFrame contains useful class of parking is: 9020


Analyse can't be continue because of there is not enough power of computational system. Complexity of the latest problem is $ O(n^2) $. In this case is aproximately $ 100000000 $ operations need to do, that will taking a lot of time. But, I found a solution: using binary search by coordinates to cuting off those parking are loacated beyond the square area. Then using <code>.distance</code> method find the distance between some parking and the building in the square area. 

Prepare df_parking for binary search by coordinates.

In [352]:
df_parking.drop(['osm_id', 'code', 'fclass', 'name'], axis=1, inplace=True)
df_parking['centroid'] = df_parking.geometry.apply(lambda polygon: polygon.centroid)
df_parking['longitude'] = df_parking.centroid.apply(lambda point: point.x)
df_parking['latitude'] = df_parking.centroid.apply(lambda point: point.y)
df_parking.head()

,geometry,centroid,longitude,latitude
0,"POLYGON ((131.89171 43.13051, 131.89171 43.130...",POINT (131.89178 43.13053),131.891775,43.130532
1,"POLYGON ((131.93530 43.11601, 131.93544 43.116...",POINT (131.93575 43.11640),131.935752,43.116398
2,"POLYGON ((131.93640 43.11494, 131.93641 43.114...",POINT (131.93655 43.11479),131.936554,43.114795
3,"POLYGON ((131.88875 43.11797, 131.88888 43.118...",POINT (131.88920 43.11795),131.889201,43.117955
4,"POLYGON ((131.88418 43.11831, 131.88420 43.118...",POINT (131.88430 43.11825),131.884297,43.118253


In [353]:
angof20 = 0.00017 #Anle in the vicinity of Vladivostok city wich corresponds to 20 meters of distance.
angof10 = 0.00008 #Anle in the vicinity of Vladivostok city wich corresponds to 10 meters of distance.
angof5 = 0.00004 #Anle in the vicinity of Vladivostok city wich corresponds to 5 meters of distance.

No. It is hopelessly. I cut off code of binary search, whatever it is didn't reduce the volume of the problem. But I can to find the bakeries nearest to some roads. Not more far than 10 meters from road. If <code>.shape[0]</code> of <code>df_bakeries</code> will be decreased well then I can do that  that I can't do now beacuse of high complexity of problem. Let will decrease the volume of problem. 

In [354]:
body = client_4b48aeaada904ad29e3d3d03072d749e.get_object(Bucket='courserads-donotdelete-pr-7suxmds10oqqm9',Key='Vladivostok_roads.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_roads = gpd.read_file(body)
df_roads.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,23734746,5113,primary,Верхнепортовая улица,,B,0,0,F,F,"LINESTRING (131.88077 43.11117, 131.88078 43.1..."
1,23745655,5114,secondary,Снеговая улица,,F,0,0,F,F,"LINESTRING (131.92678 43.13652, 131.92699 43.1..."
2,23745900,5115,tertiary,Енисейская улица,,B,0,0,F,F,"LINESTRING (131.90963 43.17322, 131.90951 43.1..."
3,23749878,5115,tertiary,,,F,0,0,F,F,"LINESTRING (131.97061 43.09137, 131.97064 43.0..."
4,23749880,5114,secondary,Сахалинская улица,,B,0,0,F,F,"LINESTRING (131.96656 43.08998, 131.96664 43.0..."


Drop roads having no name.

In [355]:
df_roads = df_roads[df_roads.name != '']

Check for unique <code>.fclass</code>.

In [356]:
pd.unique(df_roads.fclass)

array(['primary', 'secondary', 'tertiary', 'primary_link', 'residential',
       'service', 'pedestrian', 'living_street', 'unclassified',
       'secondary_link', 'trunk', 'footway', 'path', 'trunk_link'],
      dtype=object)

Needs left only roads with well traffic and check for how much the problem is decreased.

In [357]:
roads = ['primary', 'secondary', 'pedestrian']
df_roads = df_roads[df_roads.fclass.apply(lambda road: road in roads)]
df_roads.shape[0]

779

Drop unsignificant columns.

In [358]:
df_roads = df_roads.geometry
df_roads.head()

0    LINESTRING (131.88077 43.11117, 131.88078 43.1...
1    LINESTRING (131.92678 43.13652, 131.92699 43.1...
4    LINESTRING (131.96656 43.08998, 131.96664 43.0...
5    LINESTRING (131.91345 43.09879, 131.91332 43.0...
6    LINESTRING (131.89652 43.09419, 131.89637 43.0...
Name: geometry, dtype: geometry

In [359]:
dist = lambda building: bool(df_roads.distance(building).min() <= angof10)
df_bakeries = df_bakeries[df_bakeries.geometry.apply(dist)]
df_bakeries.head()

,geometry,centroid
37,"POLYGON ((131.89182 43.11467, 131.89183 43.114...",POINT (131.89222 43.11473)
67,"POLYGON ((131.93673 43.10456, 131.93685 43.104...",POINT (131.93716 43.10455)
141,"POLYGON ((131.89036 43.11510, 131.89042 43.115...",POINT (131.89014 43.11507)
274,"POLYGON ((131.92765 43.11230, 131.92838 43.112...",POINT (131.92802 43.11227)
280,"POLYGON ((131.92612 43.11198, 131.92614 43.112...",POINT (131.92635 43.11211)


In [360]:
df_bakeries.shape[0]

97

Filter out too far bakeries from parking.

In [361]:
dist = lambda building: bool(df_parking.distance(building).min() <= angof20)
df_bakeries = df_bakeries[df_bakeries.geometry.apply(dist)]
df_bakeries.reset_index(inplace=True, drop=True)
df_bakeries.head()

,geometry,centroid
0,"POLYGON ((131.89036 43.11510, 131.89042 43.115...",POINT (131.89014 43.11507)
1,"POLYGON ((131.92765 43.11230, 131.92838 43.112...",POINT (131.92802 43.11227)
2,"POLYGON ((131.92612 43.11198, 131.92614 43.112...",POINT (131.92635 43.11211)
3,"POLYGON ((131.88010 43.11662, 131.88014 43.116...",POINT (131.88047 43.11660)
4,"POLYGON ((131.90792 43.11526, 131.90816 43.115...",POINT (131.90836 43.11532)


In [362]:
df_bakeries.shape[0]

63

In [363]:
relevant_map = folium.Map(location=[latitude, longitude], zoom_start=12) #Generate map centred around the 
                                                                         #Vladivostok city.

#Add the bakeries to the map.
for lat, lng in zip(df_bakeries.centroid.y, df_bakeries.centroid.x):
    folium.features.CircleMarker([lat, lng],
                                 radius=5,
                                 color='green',
                                 fill = True,
                                 fill_color='green',
                                 fill_opacity=0.6).add_to(relevant_map)

#Display map.
relevant_map

There are found 63 places suitable to open bakeries there. As we can see all of the places is found near the road. But, some of them located are in questionable points. Using Foursquare API assign an adress string to each point, then take full details about each one. Using this information leave only relevant places.

## Relevant Places

### Assign the address

Make a function which takes a point and returns an address in a standard form. We will need packages: <code>re</code> for finding patterns in the strings and <code>time</code> for doing a time delay during waiting the response.

In [364]:
import re
from time import sleep



pattern_ul = '\sулица$'
pattern_pr = '\sпроспект$'
lpattern_ul = '^улица\s'
lpattern_pr = '^проспект\s'
def get_address(point):
    sleep(3)
    building_street = geolocator.reverse([point.y, point.x], addressdetails=True).address.split(', ')[:2]
    if building_street[1].find(' улица') != -1:
        building_street[1] = re.sub(pattern_ul, '', building_street[1])
        building_street = building_street[::-1]
        building_street.insert(0, 'ул.')
        building_street.insert(2, 'д.')
    elif building_street[1].find(' проспект') != -1:
        building_street[1] = re.sub(pattern_pr, '', building_street[1])
        building_street = building_street[::-1]
        building_street.insert(0, 'пр.')
        building_street.insert(2, 'д.')
        
    if building_street[1].find('улица ') != -1:
        building_street[1] = re.sub(lpattern_ul, '', building_street[1])
        building_street = building_street[::-1]
        building_street.insert(0, 'ул.')
        building_street.insert(2, 'д.')
    elif building_street[1].find('проспект ') != -1:
        building_street[1] = re.sub(lpattern_pr, '', building_street[1])
        building_street = building_street[::-1]
        building_street.insert(0, 'пр.')
        building_street.insert(2, 'д.')
        
    return building_street

Let apply <code>get_address</code> function.

In [365]:
df_bakeries['address'] = df_bakeries.centroid.apply(get_address)
df_bakeries.head()

,geometry,centroid,address
0,"POLYGON ((131.89036 43.11510, 131.89042 43.115...",POINT (131.89014 43.11507),"[ул., Светланская, д., 41]"
1,"POLYGON ((131.92765 43.11230, 131.92838 43.112...",POINT (131.92802 43.11227),"[ул., Светланская, д., 116]"
2,"POLYGON ((131.92612 43.11198, 131.92614 43.112...",POINT (131.92635 43.11211),"[ул., Светланская, д., 114]"
3,"POLYGON ((131.88010 43.11662, 131.88014 43.116...",POINT (131.88047 43.11660),"[ул., Светланская, д., 12]"
4,"POLYGON ((131.90792 43.11526, 131.90816 43.115...",POINT (131.90836 43.11532),"[ул., Светланская, д., 109]"


There are found several addresses such as 'Авторынок "На Снеговой", Снеговая улица', these are car market, not a good place to open a bakehouse. Exclude all places having this address.

In [366]:
df_bakeries = df_bakeries[df_bakeries.address.apply(lambda address: address != ['ул.', 'Снеговая', 'д.', 'Авторынок "На Снеговой"'])]
df_bakeries.head()

,geometry,centroid,address
0,"POLYGON ((131.89036 43.11510, 131.89042 43.115...",POINT (131.89014 43.11507),"[ул., Светланская, д., 41]"
1,"POLYGON ((131.92765 43.11230, 131.92838 43.112...",POINT (131.92802 43.11227),"[ул., Светланская, д., 116]"
2,"POLYGON ((131.92612 43.11198, 131.92614 43.112...",POINT (131.92635 43.11211),"[ул., Светланская, д., 114]"
3,"POLYGON ((131.88010 43.11662, 131.88014 43.116...",POINT (131.88047 43.11660),"[ул., Светланская, д., 12]"
4,"POLYGON ((131.90792 43.11526, 131.90816 43.115...",POINT (131.90836 43.11532),"[ул., Светланская, д., 109]"


### Assign corresponding categories and count them for each address

We need a function which takes a row from <code>df_bakeries</code> (a row represents a building) and returns a set of categories of places within this building.

In [367]:
def categories(building):
    lon = building[0].x
    lat = building[0].y

    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lon)
    results = requests.get(url).json()
    venue = results['response']['venues']
    
    #Tranform venues into a dataframe.
    df_venue = json_normalize(venue)

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in df_venue.columns if col.startswith('location.')] + ['id']
    df_venue = df_venue.loc[:, filtered_columns]

    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

    # filter the category for each row
    df_venue['categories'] = df_venue.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    df_venue.columns = [column.split('.')[-1] for column in df_venue.columns]
    df_venue = df_venue[['categories', 'address', 'id']]
    df_venue.dropna(inplace=True)

    pattern = re.compile('(({number}).+({street})|({street}).+({number})$)'.format(number=building[1], street=building[2]))
    relevant = lambda address: bool(re.match(pattern, address))
    df_venue = df_venue[df_venue.address.apply(relevant)]
    
    return df_venue

Get three lists: among of places within buildings, sets of setegories (and sets of IDs).

In [390]:
pattern = re.compile('\d+')
is_int = lambda address: bool(re.match(pattern, address[-1]))
df_bakeries = df_bakeries[df_bakeries.address.apply(is_int)]

pattern = re.compile('ул.')
is_ul = lambda address: bool(re.match(pattern, address[0]))
df_bakeries = df_bakeries[df_bakeries.address.apply(is_ul)]
df_bakeries.head()

,geometry,centroid,address
0,"POLYGON ((131.89036 43.11510, 131.89042 43.115...",POINT (131.89014 43.11507),"[ул., Светланская, д., 41]"
1,"POLYGON ((131.92765 43.11230, 131.92838 43.112...",POINT (131.92802 43.11227),"[ул., Светланская, д., 116]"
2,"POLYGON ((131.92612 43.11198, 131.92614 43.112...",POINT (131.92635 43.11211),"[ул., Светланская, д., 114]"
3,"POLYGON ((131.88010 43.11662, 131.88014 43.116...",POINT (131.88047 43.11660),"[ул., Светланская, д., 12]"
4,"POLYGON ((131.90792 43.11526, 131.90816 43.115...",POINT (131.90836 43.11532),"[ул., Светланская, д., 109]"


In [391]:
coord = df_bakeries.centroid
numbers = map(lambda row: row[3], df_bakeries.address)
street = map(lambda row: row[1], df_bakeries.address)
bakeries = zip(coord, numbers, street)

org_list = []
catg_set = []
#id_set = []
df = pd.DataFrame()
for bakehouse in bakeries:
    df = categories(bakehouse)
    org_list.append(df.shape[0])
    catg_set.append(set(df.categories))
    #id_set.append(set(df.id))

Transform each list to <code>pandas.core.series.Series</code>, join them to <code>df_bakeries</code>.

In [392]:
sr_org = pd.Series(org_list)
sr_catg = pd.Series(catg_set)
df_bakeries['organizationCount'] = sr_org
df_bakeries['categorySet'] = sr_catg
#df_bakeries['idSet'] = id_set
df_bakeries.dropna(inplace=True)
df_bakeries.organizationCount.astype('int8')
df_bakeries = df_bakeries[df_bakeries.organizationCount.apply(lambda n: n > 0)]
df_bakeries

,geometry,centroid,address,organizationCount,categorySet
0,"POLYGON ((131.89036 43.11510, 131.89042 43.115...",POINT (131.89014 43.11507),"[ул., Светланская, д., 41]",4.0,"{Bank, Monument / Landmark, Post Office}"
1,"POLYGON ((131.92765 43.11230, 131.92838 43.112...",POINT (131.92802 43.11227),"[ул., Светланская, д., 116]",2.0,"{Miscellaneous Shop, Salon / Barbershop}"
2,"POLYGON ((131.92612 43.11198, 131.92614 43.112...",POINT (131.92635 43.11211),"[ул., Светланская, д., 114]",1.0,{Pharmacy}
3,"POLYGON ((131.88010 43.11662, 131.88014 43.116...",POINT (131.88047 43.11660),"[ул., Светланская, д., 12]",1.0,{Pie Shop}
4,"POLYGON ((131.90792 43.11526, 131.90816 43.115...",POINT (131.90836 43.11532),"[ул., Светланская, д., 109]",1.0,{Wine Bar}
7,"POLYGON ((131.87846 43.11080, 131.87851 43.110...",POINT (131.87846 43.11053),"[ул., Посьетская, д., 19]",1.0,{Arcade}
8,"POLYGON ((131.92151 43.15709, 131.92166 43.157...",POINT (131.92184 43.15676),"[ул., Постышева, д., 39]",1.0,{Salon / Barbershop}
9,"POLYGON ((131.91157 43.11505, 131.91161 43.115...",POINT (131.91169 43.11507),"[ул., Пушкинская, д., 85]",1.0,{Salon / Barbershop}
13,"POLYGON ((131.87643 43.10765, 131.87646 43.107...",POINT (131.87631 43.10758),"[ул., Бестужева, д., 20]",1.0,{Office}
45,"POLYGON ((131.87643 43.10765, 131.87646 43.107...",POINT (131.87652 43.10769),"[ул., Бестужева, д., 20 с1]",5.0,"{Korean Restaurant, Lounge, Bed & Breakfast, C..."


Convert the lists which contain the addresses to the strings. No use <code>.join</code> method, because of in this cause needs more individual solution. 

In [393]:
df_bakeries['address'] = df_bakeries.address.apply(lambda address: address[0] + ' ' + address[1] + ', ' + address[2] + ' ' + address[3])
df_bakeries.reset_index(inplace=True, drop=True)
df_bakeries

,geometry,centroid,address,organizationCount,categorySet
0,"POLYGON ((131.89036 43.11510, 131.89042 43.115...",POINT (131.89014 43.11507),"ул. Светланская, д. 41",4.0,"{Bank, Monument / Landmark, Post Office}"
1,"POLYGON ((131.92765 43.11230, 131.92838 43.112...",POINT (131.92802 43.11227),"ул. Светланская, д. 116",2.0,"{Miscellaneous Shop, Salon / Barbershop}"
2,"POLYGON ((131.92612 43.11198, 131.92614 43.112...",POINT (131.92635 43.11211),"ул. Светланская, д. 114",1.0,{Pharmacy}
3,"POLYGON ((131.88010 43.11662, 131.88014 43.116...",POINT (131.88047 43.11660),"ул. Светланская, д. 12",1.0,{Pie Shop}
4,"POLYGON ((131.90792 43.11526, 131.90816 43.115...",POINT (131.90836 43.11532),"ул. Светланская, д. 109",1.0,{Wine Bar}
5,"POLYGON ((131.87846 43.11080, 131.87851 43.110...",POINT (131.87846 43.11053),"ул. Посьетская, д. 19",1.0,{Arcade}
6,"POLYGON ((131.92151 43.15709, 131.92166 43.157...",POINT (131.92184 43.15676),"ул. Постышева, д. 39",1.0,{Salon / Barbershop}
7,"POLYGON ((131.91157 43.11505, 131.91161 43.115...",POINT (131.91169 43.11507),"ул. Пушкинская, д. 85",1.0,{Salon / Barbershop}
8,"POLYGON ((131.87643 43.10765, 131.87646 43.107...",POINT (131.87631 43.10758),"ул. Бестужева, д. 20",1.0,{Office}
9,"POLYGON ((131.87643 43.10765, 131.87646 43.107...",POINT (131.87652 43.10769),"ул. Бестужева, д. 20 с1",5.0,"{Korean Restaurant, Lounge, Bed & Breakfast, C..."


Show suitable places on the map.

In [394]:
relevant_map = folium.Map(location=[latitude, longitude], zoom_start=12) #Generate map centred around the 
                                                                         #Vladivostok city.

#Add the bakeries to the map.
for lat, lng, label in zip(df_bakeries.centroid.y, df_bakeries.centroid.x, df_bakeries.address):
    folium.features.CircleMarker([lat, lng],
                                 radius=5,
                                 color='green',
                                 fill = True,
                                 fill_color='green',
                                 popup = label,
                                 fill_opacity=0.6).add_to(relevant_map)

#Display map.
relevant_map

## Conclusion

Found 10 buildings are well suitable to open a bakehouse. 

During the analyse I've faced two difficulties:
+ Not enough computational power for such as this analyse

+ Not enough accuracy of Foursquare API, the response set of venues is badly corresponding to request